### Main objectives of notebook 

- Demonstrating set up of the project
- Prerequisites: anaconda environment manager

You can run the follow commands to set up the project environment `iot`

In [ ]:
!conda create -n iot python=3.8.1 -y

In [ ]:
!conda init bash

Restart the notebook here

In [ ]:
!conda activate iot-test

In [ ]:
!pip install poetry

In [ ]:
!poetry install

Check that the libraries are properly installed 

In [ ]:
!conda list

We import the necessary libraries 

In [1]:
import os
import boto3
import getpass
from src.config import PATHS
from src.helpers import (check_s3_bucket, generate_result_from_query, setup_default_clients)

Set up credentials which will be used to access the AWS project

In [ ]:
os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass()
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass()
os.environ["AWS_DEFAULT_REGION"] = "ap-southeast-1"

Load in the environment variables as variables

In [2]:
AWS_DEFAULT_REGION = os.environ["AWS_DEFAULT_REGION"]
AWS_ACCESS_KEY_ID = os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRET_ACCESS_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]

Test that the S3 and athena clients are running by running the helper function

In [3]:
s3, athena = setup_default_clients()

Check that our default S3 bucket exists by setting up a `resource` session

In [4]:
PROJECT_BUCKET = "smu-is614-iot-step-tracker"

In [5]:
s3_resource = boto3.resource(
    "s3",
    region_name=AWS_DEFAULT_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

In [6]:
print(
check_s3_bucket(
    s3_resource=s3_resource, 
    bucket_name=PROJECT_BUCKET
))

Bucket Exists
True


List folders in s3 bucket


In [7]:
response = s3.list_objects(Bucket=PROJECT_BUCKET, Delimiter="/")
print("\n".join(content.get("Prefix") for content in response["CommonPrefixes"]))

Unsaved/
athena/
demo/
inference/
microbit/
mlflow-artifact-store/
models/
package/
queries/


Test the athena client by setting up a `TESTING_QUERY`

In [8]:
TESTING_QUERY = """
    SELECT
        *,
        case when uuid like '%_walk_%' then true else false end as target
    FROM
        "smu-iot"."microbit"
    WHERE
        seconds IS NOT null 
      AND uuid = 'licheng_walk_2'
    ORDER BY
        uuid, timestamp, seconds
"""

We can send a query and get a response using the function below

In [9]:
response = generate_result_from_query(
    athena_client=athena,
    query_string=TESTING_QUERY,
    database="smu-iot"
)

['1677075254', '0.94', 'licheng_walk_2', '1.0', '0.0', '0.0', '-10.0', '70.0', '-69.0', 'true']
['1677075254', '1.01', 'licheng_walk_2', '0.0', '0.0', '0.0', '-14.0', '66.0', '-66.0', 'true']
['1677075254', '1.08', 'licheng_walk_2', '-1.0', '-1.0', '0.0', '-11.0', '70.0', '-72.0', 'true']
['1677075254', '1.21', 'licheng_walk_2', '-1.0', '0.0', '0.0', '-10.0', '67.0', '-72.0', 'true']
['1677075254', '1.34', 'licheng_walk_2', '-1.0', '-1.0', '0.0', '-17.0', '60.0', '-70.0', 'true']
['1677075254', '1.4', 'licheng_walk_2', '-1.0', '-1.0', '1.0', '-20.0', '64.0', '-73.0', 'true']
['1677075254', '1.47', 'licheng_walk_2', '-5.0', '-5.0', '5.0', '-24.0', '53.0', '-64.0', 'true']
['1677075254', '1.54', 'licheng_walk_2', '-5.0', '-5.0', '5.0', '-31.0', '66.0', '-60.0', 'true']
['1677075254', '1.6', 'licheng_walk_2', '-2.0', '-3.0', '8.0', '-39.0', '65.0', '-54.0', 'true']
['1677075254', '1.67', 'licheng_walk_2', '-3.0', '-2.0', '10.0', '-44.0', '78.0', '-55.0', 'true']
['1677075254', '1.73', 'li

In [10]:
print(response[0:5])

[{'timestamp': '1677075254', 'seconds': '0.94', 'uuid': 'licheng_walk_2', 'gyro_x': '1.0', 'gyro_y': '0.0', 'gyro_z': '0.0', 'accel_x': '-10.0', 'accel_y': '70.0', 'accel_z': '-69.0', 'target': 'true'}, {'timestamp': '1677075254', 'seconds': '1.01', 'uuid': 'licheng_walk_2', 'gyro_x': '0.0', 'gyro_y': '0.0', 'gyro_z': '0.0', 'accel_x': '-14.0', 'accel_y': '66.0', 'accel_z': '-66.0', 'target': 'true'}, {'timestamp': '1677075254', 'seconds': '1.08', 'uuid': 'licheng_walk_2', 'gyro_x': '-1.0', 'gyro_y': '-1.0', 'gyro_z': '0.0', 'accel_x': '-11.0', 'accel_y': '70.0', 'accel_z': '-72.0', 'target': 'true'}, {'timestamp': '1677075254', 'seconds': '1.21', 'uuid': 'licheng_walk_2', 'gyro_x': '-1.0', 'gyro_y': '0.0', 'gyro_z': '0.0', 'accel_x': '-10.0', 'accel_y': '67.0', 'accel_z': '-72.0', 'target': 'true'}, {'timestamp': '1677075254', 'seconds': '1.34', 'uuid': 'licheng_walk_2', 'gyro_x': '-1.0', 'gyro_y': '-1.0', 'gyro_z': '0.0', 'accel_x': '-17.0', 'accel_y': '60.0', 'accel_z': '-70.0', 'ta